## gpu 확인 ##

```
사용환경
우분투 맥 윈도우
cuda,cudnn 설치
파이썬 3.7~3.10, 아나콘다 가상환경
```



In [ ]:
!nvidia-smi

Tue Nov 21 13:11:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup 라이브러리 불러오기

In [ ]:
!pip install -q peft transformers datasets evaluate
!pip install wandb

In [ ]:
!pip install einops

In [ ]:
# 모델
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM
)
# 파인튜닝
from peft import (
    LoraConfig,
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
)
# 파이토치 데이터셋
from datasets import load_dataset
import evaluate
import torch
import os

model_name_or_path = "klue/roberta-small"
# model_name_or_path = "beomi/kobert"






## Dataset


```
로컬 환경 시 해당 경로 사용
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/dataset"

/content/drive/MyDrive/dataset


In [ ]:
import pandas as pd
df = pd.read_csv("./review_tags.csv", encoding="UTF-8", index_col=False)
id = pd.read_csv("./review_tag_types.csv", encoding="UTF-8", index_col=False)
df = df.loc[:,{'brand_code','review_id','message','review_tag_type_id','sentiment'}]

df = df[df['brand_code'] == "topten10mall.com"]

<ipython-input-6-553d5e0000fb>:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df.loc[:,{'brand_code','review_id','message','review_tag_type_id','sentiment'}]


In [ ]:
id = id[id['visible']==1]
id = id[id['nlp_category_type_id']==2]
id = id.loc[:,{'id','name'}]
df, id

<ipython-input-7-c745ba1386d1>:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  id = id.loc[:,{'id','name'}]


(               brand_code  review_id  sentiment  \
 0        topten10mall.com     838412          1   
 1        topten10mall.com     838412          1   
 2        topten10mall.com     838412          0   
 3        topten10mall.com     838412          2   
 4        topten10mall.com     865458          1   
 ...                   ...        ...        ...   
 3539497  topten10mall.com    2047424          1   
 3539498  topten10mall.com    2047425          1   
 3539499  topten10mall.com    2047426          1   
 3539500  topten10mall.com    2047426          1   
 3539501  topten10mall.com    2047426          1   
 
                                                    message  review_tag_type_id  
 0        한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...                  28  
 1        한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...                  18  
 2        한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...                  25  
 3        한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸

In [ ]:
id = id.rename(columns={'id':'review_tag_type_id' , 'name' : 'keyword'})
new_df = pd.merge(df, id, on = "review_tag_type_id")
new_df = new_df.drop({'review_tag_type_id','brand_code'},axis = 1)

new_df.isna().sum()

review_id    0
sentiment    0
message      0
keyword      0
dtype: int64

In [ ]:
new_df = new_df[new_df['message'].notna() ]

In [ ]:
new_df = new_df.sort_values(by ='review_id').reset_index(drop = True)

In [ ]:
new_df['sentiment'] = new_df['sentiment'].replace(2,4).replace(1,3).replace(0,2).replace(-1,1).replace(-2,0)


In [ ]:
 new_df['label'] = [ new_df['keyword'][x] + str(new_df['sentiment'][x]) for x in range(len(new_df)) ]

In [ ]:
new_df['message2'] = [ "### review: "+new_df['message'][x]+" ### keyword: "+new_df['keyword'][x]  for x in range(len(new_df)) ]

new_df

,review_id,sentiment,message,keyword,label,message2
0,821865,3,핏 자체도 이쁘지만 색이 너무 잘 바졌어요 ㅎ 강추합니다,라인(핏),라인(핏)3,### review: 핏 자체도 이쁘지만 색이 너무 잘 바졌어요 ㅎ 강추합니다 #...
1,821865,4,핏 자체도 이쁘지만 색이 너무 잘 바졌어요 ㅎ 강추합니다,색상,색상4,### review: 핏 자체도 이쁘지만 색이 너무 잘 바졌어요 ㅎ 강추합니다 #...
2,821866,3,"편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다 허리 둘레가 좀 커서...",기능성,기능성3,"### review: 편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다..."
3,821866,3,"편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다 허리 둘레가 좀 커서...",소재,소재3,"### review: 편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다..."
4,821866,3,"편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다 허리 둘레가 좀 커서...",착용감,착용감3,"### review: 편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다..."
...,...,...,...,...,...,...
3509546,2047424,3,블루로 구매후 만족스러워 블랙진도 구매합니다. 핏도 좋고 편합니다. 기장은 쪼금 짧...,착용감,착용감3,### review: 블루로 구매후 만족스러워 블랙진도 구매합니다. 핏도 좋고 편합...
3509547,2047425,3,도톰해서 한여름 야외에선 어렵고 에어컨 있는 실내나 여름 전후에 입기 매우 좋습니다...,소재,소재3,### review: 도톰해서 한여름 야외에선 어렵고 에어컨 있는 실내나 여름 전후...
3509548,2047426,3,많이 두껍지 않은 적당한 두께감에 부드러운 감촉이 느껴지는 양말입니다. 아들이 답답...,신축성,신축성3,### review: 많이 두껍지 않은 적당한 두께감에 부드러운 감촉이 느껴지는 양...
3509549,2047426,3,많이 두껍지 않은 적당한 두께감에 부드러운 감촉이 느껴지는 양말입니다. 아들이 답답...,착용감,착용감3,### review: 많이 두껍지 않은 적당한 두께감에 부드러운 감촉이 느껴지는 양...


In [ ]:
new_df = new_df.iloc[0:100000,:].reset_index(drop=True)

In [ ]:
categories = new_df['label'].unique()
train_df = pd.DataFrame()
test_df = pd.DataFrame()


for cat in categories:
    category_data = new_df[new_df['label'] == cat]
    train_size = int(len(category_data) * 0.7)
    train_data = category_data.sample(n=train_size, replace=True, random_state=42)
    test_data = category_data.drop(train_data.index)

    train_df = pd.concat([train_df, train_data])
    test_df = pd.concat([test_df, test_data])

print(train_df.head())
print(test_df.head())
print(f"트레인 라벨 수: {train_df['label'].unique().__len__()}")
print(f"테스트 라벨 수: {test_df['label'].unique().__len__()}")

       review_id  sentiment  \
15645     827145          3   
99342     855665          3   
96057     854550          3   
95251     854305          3   
68805     845227          3   

                                                 message keyword   label  \
15645  여성 쿨에어 브래패드 캐미솔 흰색으로 구매했습니다^^\n건조기에 돌릴거 생각해서 사...   라인(핏)  라인(핏)3   
99342  ㄹ 리뷰를 보니. 조금크다 또는 조금작다 라고 해서..\n고민끝에 하프팬츠 4개 동...   라인(핏)  라인(핏)3   
96057  이제품만 따로늦게 도착했는데 기다린 보람이있네요.약간 오버핏으로 약간여유가 있어서 ...   라인(핏)  라인(핏)3   
95251  배송도 빠르고 좋습니다. 옷이 딱 맞아서 불편하지 않네요. 앞으로도 좋은옷 많이 생...   라인(핏)  라인(핏)3   
68805  주문을 여러게해서  그런지 다른제품은 담날 도착했고 바지는 4일후쯤 도착했어요\n배...   라인(핏)  라인(핏)3   

                                                message2  
15645  ### review: 여성 쿨에어 브래패드 캐미솔 흰색으로 구매했습니다^^\n건조기...  
99342  ### review: ㄹ 리뷰를 보니. 조금크다 또는 조금작다 라고 해서..\n고민...  
96057  ### review: 이제품만 따로늦게 도착했는데 기다린 보람이있네요.약간 오버핏으...  
95251  ### review: 배송도 빠르고 좋습니다. 옷이 딱 맞아서 불편하지 않네요. 앞...  
68805  ### review: 주문을 여러게해서  그런지 다른제품은 담날 도착했고 바지는 4...  
  

In [ ]:
train_df = train_df.loc[:,['message2','sentiment']].reset_index(drop = True)
test_df = test_df.loc[:,['message2','sentiment']].reset_index(drop = True)

In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

## 전처리

In [ ]:
#공백 위치 설정
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"
#토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id  # 여백 토큰과 end of sentence token




# 사이즈 안맞을 경우(polyglot 사용시)
# CUTOFF_LEN = 2048
# def tokenize(prompt, add_eos_token=True):
#     result = tokenizer(
#         prompt,
#         truncation=True,
#         max_length=CUTOFF_LEN,
#         padding=False,
#         return_tensors=None,
#     )
#     if (
#         result["input_ids"][-1] != tokenizer.eos_token_id
#         and len(result["input_ids"]) < CUTOFF_LEN
#         and add_eos_token
#     ):
#         result["input_ids"].append(tokenizer.eos_token_id)
#         result["attention_mask"].append(1)

#     result["labels"] = result["input_ids"].copy()

#     return result


def tokenize_function(examples):
    # max_length=None #=> use the model max length (it's actually the default)
    outputs = tokenizer(examples["message2"], truncation=True)
    return outputs

In [ ]:
tokenized_datasets = dataset.shuffle(seed=42).map(
    tokenize_function,
    batched=True,
    remove_columns=["message2"],)  #message컬럼은 불필요

tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")

Map:   0%|          | 0/69966 [00:00<?, ? examples/s]

Map:   0%|          | 0/50263 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 69966
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50263
    })
})

## peft, 모델 설정

In [ ]:
# LORA_R = 128 #8 the rank of the update matrices, Lower rank results in smaller update matrices with fewer trainable parameters.
# LORA_ALPHA = 256 #  LoRA scaling factor.
# LORA_DROPOUT= 0.3  #0.2 The modules (for example, attention blocks) to apply the LoRA update matrices.
# LORA_TARGET_MODULES = ["query","value"]
# LORA_TARGET_MODULES = [ "q_proj", "v_proj" ] #llama
# LORA_TARGET_MODULES = ["query_key_value"] #causal



In [ ]:
# peft_config = LoraConfig(
#     r=LORA_R,
#     lora_alpha=LORA_ALPHA,
#     target_modules=LORA_TARGET_MODULES,
#     lora_dropout=LORA_DROPOUT,
#     bias="none",
#     task_type="SEQ_CLS",
# )


In [ ]:
#Data collators are objects that will form a batch by using a list of dataset elements as input.
#These elements are of the same type as the elements of train_dataset or eval_dataset.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
id2label = {0: "매우부정", 1: "부정", 2: "중립", 3: "긍정", 4: "매우긍정"}
label2id = {"매우부정": 0, "부정": 1, "중립": 2, "긍정": 3, "매우긍정": 4}


model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=5, id2label=id2label, label2id=label2id
    # ,ignore_mismatched_sizes=True
)

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc=accuracy_score(labels , preds)
  return {"accuracy":acc, "f1": f1}

## Training

In [ ]:
!wandb login

wandb: Currently logged in as: jeongdongseon. Use `wandb login --relogin` to force relogin


In [ ]:
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="cllama"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [ ]:
# from transformers.optimization import Adafactor, AdafactorSchedule

# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-5,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.1,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False,
# )

# lr_scheduler = AdafactorSchedule(optimizer)


In [ ]:
training_args = TrainingArguments(
    output_dir="dongseon/bert_test",
    learning_rate= 1e-5, #5e-4
    per_device_train_batch_size=64, #64
    per_device_eval_batch_size=64,  #16
    num_train_epochs=5,
    optim="adamw_hf",
    adam_beta1 = 0.9, #( float, 선택 사항 , 기본값은 0.9) — AdamW 최적화 프로그램 의 beta1  매개변수
    adam_beta2 = 0.999, #( float, 선택 사항 , 기본값은 0.999) — AdamW 최적화 프로그램 의 beta2  매개변수
    adam_epsilon= 1e-8, # ( float, 선택 사항 , 기본값은 1e-8) — AdamW 최적화 프로그램 의 엡실론  매개변수
    weight_decay= 0.5 , #가중치 감소, 커질수록 오버피팅 방지 0.1
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb",


    # eval_accumulation_steps  = 2, # (더 빠르지만 더 많은 메모리가 필요함).
    # gradient_accumulation_steps  = 128//32 ,  # 역방향/업데이트 단계를 수행하기 전에 기울기를 누적할 업데이트 단계 수
    # max_grad_norm= 4, # ( float, 선택 사항 , 기본값은 1.0) — 최대 그라데이션 표준(그라디언트 클리핑용).
    # fp16 = True, # ( bool, 선택 사항 , 기본값은 False) — 32비트 훈련 대신 fp16 16비트(혼합) 정밀도 훈련을 사용할지
    # label_smoothing_factor = 0.1, # ( float, 선택 사항 , 기본값은 0.0) — 원핫 인코딩 레이블이 각각 0과 1에서 label_smoothing_factor/num_labels,,,label_smoothing_factor + label_smoothing_factor/num_labels.

)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    # optimizers=(optimizer, lr_scheduler),
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: jeongdongseon. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.352400,0.316974,0.878380,0.878217
2,0.236600,0.267585,0.899966,0.900217
3,0.187700,0.255174,0.912082,0.910507
4,0.151900,0.262508,0.912262,0.912347
5,0.134100,0.261346,0.915146,0.914825


TrainOutput(global_step=5470, training_loss=0.23543010899963937, metrics={'train_runtime': 1334.4462, 'train_samples_per_second': 262.154, 'train_steps_per_second': 4.099, 'total_flos': 1.253027569576446e+16, 'train_loss': 0.23543010899963937, 'epoch': 5.0})

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

## 모델 업로드

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [ ]:
model_output_dir = "./"
model.save_pretrained(model_output_dir)

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
model.push_to_hub("dongseon/cllama_sentiment", create_pr=1,use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/272M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dongseon/cllama_sentiment/commit/52363e54364d8ce06bb79403bfdd46d857536e19', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='52363e54364d8ce06bb79403bfdd46d857536e19', pr_url='https://huggingface.co/dongseon/cllama_sentiment/discussions/7', pr_revision='refs/pr/7', pr_num=7)

## 테스트

In [ ]:
# id2label_sent = {0: "매우부정", 1: "부정", 2: "중립", 3: "긍정", 4: "매우긍정"}
# label2id_sent = {"매우부정": 0, "부정": 1, "중립": 2, "긍정": 3, "매우긍정": 4}

# model_name = 'klue/roberta-small'

# sent_name = "dongseon/cllama_sentiment"
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name
#     , num_labels=5, id2label=id2label_sent, label2id=label2id_sent
# )
# if any(k in model_name for k in ("gpt", "opt", "bloom","poly")):
#     padding_side = "left"
# else:
#     padding_side = "right"

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
# if getattr(tokenizer, "pad_token_id") is None:
#     tokenizer.pad_token_id = tokenizer.eos_token_id  # 여백 토큰과 end of sentence token


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

**로컬 모델 사용하기**

In [ ]:
# txt = "흰옷이라서 속이 좀 비치지만 라인이 이뻐요"
txt = "단추가 덜렁거려요. 그래도 배송은 빨랐어요."
# txt = "화면에서는 하늘색인데 진한 파랑색입니다.. 근데 세탁 후 보풀이 생겼어요"

# txt = "생각보다 더 따뜻하고 프린팅도 귀엽고 색감도 이쁘고 잘산것 같아요"
key = "마감처리"
inputs = tokenizer(

        f'### review: {txt} ### keyword: {key} '

    , return_tensors="pt")

In [ ]:

model.eval()
with torch.no_grad():
     # 모델이 있는 디바이스로 입력 데이터 이동
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    # 모델 호출
    logits = model(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
keyword = model.config.id2label[predicted_class_id]
print(keyword)

부정
